In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import pathlib
from pathlib import Path
import torch
sns.set_style('white')
from skimage.io import imread, imsave
import scipy.stats
%matplotlib qt5

In [3]:
DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/membrane_dots/EXP-22-BQ4393/Run/Pos11/Pos0/fluor580/')

In [4]:
fluor_filenames = sorted(list(DATA_DIR.glob('*.tiff')))

In [5]:
h, w = imread(fluor_filenames[0]).shape

In [6]:
image_stack = np.zeros((len(fluor_filenames), h, w), dtype='uint16')
for i in range(len(fluor_filenames)):
    image_stack[i,:,:] = imread(fluor_filenames[i])

In [7]:
image_stack.shape

(1000, 46, 338)

In [10]:
max_values = []
min_values = []

for i in range(1000):
    max_values.append(image_stack[i].max())
    min_values.append(image_stack[i].min())

In [14]:
max_values

[374,
 393,
 634,
 491,
 330,
 324,
 303,
 324,
 306,
 314,
 294,
 339,
 319,
 332,
 301,
 309,
 339,
 318,
 302,
 296,
 289,
 277,
 288,
 302,
 297,
 273,
 282,
 294,
 327,
 284,
 276,
 294,
 266,
 264,
 282,
 265,
 271,
 295,
 261,
 293,
 308,
 314,
 285,
 262,
 266,
 254,
 252,
 267,
 263,
 255,
 270,
 285,
 279,
 276,
 256,
 256,
 281,
 246,
 254,
 261,
 273,
 260,
 284,
 241,
 259,
 254,
 262,
 277,
 267,
 249,
 283,
 286,
 255,
 332,
 273,
 283,
 270,
 281,
 340,
 271,
 287,
 283,
 250,
 265,
 270,
 290,
 286,
 274,
 279,
 295,
 316,
 277,
 309,
 273,
 286,
 240,
 300,
 280,
 310,
 276,
 298,
 311,
 272,
 314,
 262,
 249,
 248,
 275,
 257,
 277,
 274,
 280,
 247,
 260,
 308,
 265,
 263,
 301,
 254,
 259,
 271,
 255,
 254,
 263,
 272,
 256,
 278,
 244,
 240,
 278,
 275,
 263,
 279,
 282,
 264,
 251,
 265,
 254,
 252,
 245,
 251,
 251,
 262,
 248,
 287,
 241,
 251,
 273,
 260,
 265,
 272,
 251,
 237,
 249,
 266,
 244,
 237,
 287,
 264,
 274,
 265,
 309,
 277,
 291,
 270,
 256,
 299

In [25]:
indices = np.where(max_values < np.percentile(max_values, 99))

In [26]:
np.mean(np.array(max_values)[indices])

259.16262626262625

In [12]:
np.percentile(max_values, 50), np.percentile(min_values, 90)

(255.0, 98.0)

In [8]:
ind = np.where(image_stack <= 0)

In [9]:
images = image_stack

In [10]:
map_empty = np.where(images.mean(0) < np.percentile(images.mean(0), 50))

In [11]:
map_empty

(array([ 0,  0,  0, ..., 45, 45, 45]),
 array([  0,   1,   2, ..., 335, 336, 337]))

In [12]:
pixel_vals = images[:, map_empty[0], map_empty[1]].reshape(-1)

In [13]:
floc = 0

In [14]:
fit_alpha, fit_loc, fit_beta = scipy.stats.gamma.fit(pixel_vals, floc=floc)

In [15]:
xlim = None
fig, axs = plt.subplot_mosaic([['a)', 'a)'], ['b)', 'c)']])
im = axs['a)'].imshow(images.mean(0))
#axs['a)'].set_title('a)', fontfamily='serif', loc='left', fontsize='small')
axs['a)'].set_title('a) Mean image of stack of 1000 images', fontstyle='italic', fontsize='small')
divider = make_axes_locatable(axs['a)'])
cax = divider.append_axes('right', size='5%',pad='2%')
cb1 = fig.colorbar(im, cax=cax)

low, high = np.min(images.mean(0)), np.max(images.mean(0))

axs['b)'].hist(images.mean(0).ravel(), bins=np.linspace(low, high, 250), histtype='step', label='Pixel distribution')
#axs['b)'].set_title('b)', fontfamily='serif', loc='left', fontsize='small')
axs['b)'].set_title('b) Distribution of mean pixel values', fontstyle='italic', fontsize='small')
axs['b)'].axvline(np.percentile(images.mean(0), 50), color='r')
axs['b)'].text(np.percentile(images.mean(0), 50), 25,'50th percentile')
if xlim is None: 
    low,high = pixel_vals.min(),pixel_vals.max()
else:
    low,high = xlim[0],xlim[1]
    
    

axs['c)'].hist(pixel_vals, bins=np.linspace(low, high, 250), histtype='step', label='data')
axs['c)'].hist(np.random.gamma(shape=fit_alpha,scale=fit_beta,size=len(pixel_vals))+floc,
                            bins=np.linspace(low,high, 250),
                            histtype='step', label='fit')
#axs['c)'].set_title('c)', fontfamily='serif', fontsize='small')
axs['c)'].set_title('c) Backgroud distribution and Fitted gamma distribution', fontstyle='italic', fontsize='small')
plt.xlim(low,high)
axs['c)'].legend()
plt.tight_layout()
plt.show()

In [16]:
fit_alpha * fit_beta

140.96716838178543

In [18]:
np.min(images.mean(0)),  np.max(images.mean(0))

(116.339, 186.416)

In [20]:
(116.330 - 100) * 0.25 / 0.95

4.2973684210526315

In [ ]:
4.297 

In [31]:
(140 - 100) * 0.25 / 0.95

10.526315789473685

In [32]:
10.52/ 0.021

500.9523809523809

In [27]:
(255  - 100) * 0.25 / 0.95

40.78947368421053

In [30]:
48.789 / 0.021

2323.285714285714

In [21]:
(140 - 100) * 0.25 / 0.95

10.526315789473685

In [ ]:
bg_photons = 10

In [18]:
5539 * 0.8

4431.2

In [19]:
8876 * 1.2

10651.199999999999

In [20]:
4431/10651

0.4160172753732044

In [23]:
images.mean(0).min()

116.339

In [32]:
np.where(images.mean(0) < 125)

(array([ 0,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,  7,
         7,  7,  7,  7,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9, 10,
        10, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13,
        13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 16, 19, 19, 21, 21, 22,
        25, 32, 33, 33, 33, 33, 34, 35, 35, 35, 35, 35, 35, 35, 35, 36, 36,
        36, 36, 36, 36, 36, 36, 37, 37, 38, 38, 38, 38, 38, 38, 39, 39, 39,
        39, 39, 40, 40, 40, 40, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42,
        43, 43, 44, 44, 45, 45, 45, 45, 45, 45, 45]),
 array([157, 287,  10,  69, 246, 257, 304,  37,  70,  87, 105, 177, 185,
        301, 311, 323, 336, 119, 183, 316, 332,  82,  92, 144, 146, 172,
        175,  12,  77,  78,  82, 141,   3,  51, 159, 169, 259, 301,  35,
         47,  84, 332,  12,  24,  34,  67, 111, 112, 208, 237,  17, 292,
         14,  18,  33,  35,  61,  66,  73, 328

In [30]:
np.mean(images[:, 44, 39])

116.971

In [22]:
images.mean(0).max()

186.416

In [42]:
factor = images.mean(0).max()

In [43]:
factor

186.416

In [44]:
offset = images.mean()

In [45]:
offset

146.72558290455365